In [39]:
import os
import re
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix

In [45]:
f1 = open("positive.txt", "r",encoding="latin") # 'r' is for reading
short_pos = f1.readlines()

In [49]:
short_pos[1]

'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth . '

In [47]:
type(short_pos)

list

In [48]:
short_pos = [re.sub("\n","", i) for i in short_pos]
x_short_pos = short_pos[:1000]

In [52]:
f2 = open("negative.txt", "r", encoding="latin")
short_neg = f2.readlines()
short_neg = [re.sub("\n","",i) for i in short_neg]
x_short_neg = short_neg[:1000]

In [53]:
print(short_neg[:1000])
print(short_pos[:1000])

['simplistic , silly and tedious . ', "it's so laddish and juvenile , only teenage boys could possibly find it funny . ", 'exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . ', '[garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation . ', 'a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification . ', "the story is also as unoriginal as they come , already having been recycled more times than i'd care to count . ", "about the only thing to give the movie points for is bravado -- to take an entirely stale concept and push it through the audience's meat grinder one more time . ", 'not so much farcical as sour . ', 'unfortunately the story and the actors are served with a hack script . ', 'all the more disquieting for its relatively gore-free allusions to the serial murders , but it falls do

In [54]:
#combine the first 1000 positive reviews and first 1000 negative reviews to form a corpus
data = x_short_pos + x_short_neg
#create the target variable representing 1000 'pos' and 'neg' instances each, wrt the data created above
target = ['pos'] *1000 + ['neg']*1000

In [55]:
len(data)

2000

In [56]:
len(target)

2000

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=1234)

In [67]:
#Get doc-term matrix on train data
cv = CountVectorizer(stop_words='english',lowercase=True, strip_accents='unicode',decode_error='ignore')
cv.fit(X_train)
print(cv.vocabulary_)
tdm_train =cv.fit_transform(X_train)
Mat = tdm_train.todense()
Mat
Mat.shape

{'day': 1361, 'great': 2494, 'bond': 621, 'movie': 3675, 'good': 2454, 'makes': 3404, 'better': 534, 'typical': 5952, 'knock': 3133, 'offs': 3892, 'storytelling': 5442, 'feels': 2128, 'slight': 5205, 'given': 2423, 'time': 5790, 'consider': 1153, 'looseness': 3332, 'piece': 4158, 'picture': 4155, 'begins': 508, 'resemble': 4697, 'shapeless': 5069, 'grasping': 2486, 'actors': 116, 'workshop': 6369, 'story': 5440, 'wraps': 6386, 'kind': 3119, 'elegant': 1780, 'symmetry': 5622, 'rare': 4530, 'film': 2164, 'today': 5806, 'warned': 6233, 'slow': 5212, 'slog': 5210, 'real': 4552, 'people': 4093, 'living': 3307, 'lives': 3306, 'concerned': 1118, 'future': 2348, 'elderly': 1774, 'mentally': 3518, 'handicapped': 2570, 'family': 2076, 'member': 3509, 'lee': 3227, 'jeong': 3046, 'hyang': 2776, 'tells': 5702, 'lovingly': 3351, 'films': 2172, 'beautifully': 496, 'couldn': 1213, 'help': 2650, 'captivated': 804, 'nicely': 3790, 'serves': 5031, 'examination': 1950, 'society': 5251, 'transition': 5857,

(1600, 6435)

In [63]:
#Get doc-term matrix on test data
tdm_test = cv.transform(X_test)
Mat_test = tdm_test.todense()
Mat_test
Mat_test.shape



(400, 6435)

In [68]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore") #just to ignore any warnings

#Training the model
log_reg = LogisticRegression()
lr_clf = log_reg.fit(tdm_train,y_train)
#Predicting on train data
train_pred = lr_clf.predict(tdm_train)
# Predicitng on test data
test_pred = lr_clf.predict(tdm_test)

In [69]:
print("Train_Confusion Matrix: \n", confusion_matrix(y_train,train_pred))
print("Test_Confusion Matrix: \n", confusion_matrix(y_test,test_pred))

Train_Confusion Matrix: 
 [[794   4]
 [  5 797]]
Test_Confusion Matrix: 
 [[145  57]
 [ 62 136]]


In [70]:
print("A glance of first 10 values:\n", "\n Test Actuals: ", y_test[:10],"\n", "Test Predictions: ", test_pred[:10])

A glance of first 10 values:
 
 Test Actuals:  ['neg', 'pos', 'neg', 'pos', 'pos', 'neg', 'neg', 'neg', 'neg', 'pos'] 
 Test Predictions:  ['neg' 'neg' 'neg' 'pos' 'pos' 'neg' 'pos' 'neg' 'neg' 'pos']


In [71]:
#Test Metrics
acc = accuracy_score(y_test, test_pred)
rec = recall_score(y_test, test_pred, pos_label='neg')
prec = precision_score(y_test, test_pred, pos_label='neg')

print("Results of logistic regression on test data:","\nAccuracy:",acc, "\nRecall:",rec, "\nPrecision:",prec)

Results of logistic regression on test data: 
Accuracy: 0.7025 
Recall: 0.7178217821782178 
Precision: 0.7004830917874396


In [72]:
#Work with any other classification model and check if you can improve the accuracies
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(tdm_train,y_train)
test_pred = classifier.predict(tdm_test)
confusion_matrix(y_test,test_pred)

array([[137,  65],
       [ 55, 143]], dtype=int64)

In [73]:
acc = accuracy_score(y_test, test_pred)
rec = recall_score(y_test, test_pred, pos_label='neg')
prec = precision_score(y_test, test_pred, pos_label='neg')

print("Results of Multinomial_NB on test data:","\nAccuracy:",acc, "\nRecall:",rec, "\nPrecision:",prec)

Results of Multinomial_NB on test data: 
Accuracy: 0.7 
Recall: 0.6782178217821783 
Precision: 0.7135416666666666
